In [48]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [49]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [50]:
model = load_model("model.h5")

with open("onehot_encoder_geo.pkl", "rb") as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

In [51]:
# Example input data
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000,
}

In [52]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data["Geography"]]]).toarray()
geo_encoded_df = pd.DataFrame(
    geo_encoded, columns=label_encoder_geo.get_feature_names_out(["Geography"])
)
geo_encoded_df

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [53]:
input_df = pd.DataFrame([input_data])
input_data = pd.concat([input_df.reset_index(drop=True), geo_encoded_df], axis=1)
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [54]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [55]:
## concatenate with one hot encoded data
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [56]:
## scaling the input data
input_scaler = scaler.transform(input_df)
input_scaler

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [57]:
## prediction
prediction = model.predict(input_scaler)
prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


array([[0.02487081]], dtype=float32)

In [59]:
prediction_probablity = prediction[0][0]
prediction_probablity

0.024870813

In [60]:
if prediction_probablity > 0.5:
    print("Customer is likely to exit the bank")
else:
    print("Customer is unlikely to exit the bank")

Customer is unlikely to exit the bank
